<a href="https://colab.research.google.com/github/Tomas-Turner/mgmt467-analytics-portfolio/blob/main/Unit2_Lab1_PromptPlusExamples_Colab_Kaggle_GCS_BQ_DQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MGMT 467 — Prompt-Driven Lab (with Commented Examples)
## Kaggle ➜ Google Cloud Storage ➜ BigQuery ➜ Data Quality (DQ)

**How to use this notebook**
- Each section gives you a **Build Prompt** to paste into Gemini/Vertex AI (or Gemini in Colab).
- Below each prompt, you’ll see a **commented example** of what a good LLM answer might look like.
- **Do not** just uncomment and run. Use the prompt to generate your own code, then compare to the example.
- After every step, run the **Verification Prompt**, and write the **Reflection** in Markdown.

> Goal today: Download the Netflix dataset (Kaggle) → Stage on GCS → Load into BigQuery → Run DQ profiling (missingness, duplicates, outliers, anomaly flags).


### Academic integrity & LLM usage
- Use the prompts here to generate your own code cells.
- Read concept notes and write the reflection answers in your own words.
- Keep credentials out of code. Upload `kaggle.json` when asked.


## Learning objectives
1) Explain **why** we stage data in GCS and load it to BigQuery.  
2) Build an **idempotent**, auditable pipeline.  
3) Diagnose **missingness**, **duplicates**, and **outliers** and justify cleaning choices.  
4) Connect DQ decisions to **business/ML impact**.


## 0) Environment setup — What & Why
Authenticate Colab to Google Cloud so we can use `gcloud`, GCS, and BigQuery. Set **PROJECT_ID** and **REGION** once for consistency (cost/latency).

### Build Prompt (paste to LLM)
You are my cloud TA. Generate a single **Colab code cell** that:
1) Authenticates to Google Cloud in Colab,  
2) Prompts for `PROJECT_ID` via `input()` and sets `REGION="us-central1"` (editable),  
3) Exports `GOOGLE_CLOUD_PROJECT`,  
4) Runs `gcloud config set project $GOOGLE_CLOUD_PROJECT`,  
5) Prints both values. Add 2–3 comments explaining what/why.
End with a comment: `# Done: Auth + Project/Region set`.


In [3]:
#EXAMPLE (from LLM) — Auth + Project/Region (commented; write your own cell using the prompt)
from google.colab import auth
auth.authenticate_user()

import os
PROJECT_ID = input("Enter your GCP Project ID: ").strip()
REGION = "us-central1"  # keep consistent; change if instructed
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
print("Project:", PROJECT_ID, "| Region:", REGION)

# Set active project for gcloud/BigQuery CLI
!gcloud config set project $GOOGLE_CLOUD_PROJECT
!gcloud config get-value project
# Done: Auth + Project/Region set

Enter your GCP Project ID: mgmt-467-35946
Project: mgmt-467-35946 | Region: us-central1
Updated property [core/project].
mgmt-467-35946


### Verification Prompt
Generate a short cell that prints the active project using `gcloud config get-value project` and echoes the `REGION` you set.


In [7]:
!gcloud config get-value project
import os
!echo $REGION

mgmt-467-35946
us-central1


**Reflection:** Why do we set `PROJECT_ID` and `REGION` at the top? What can go wrong if we don’t?

We need to set the Project ID and Region for a few different reasons. First, for consistency, doing this will ensure that all of the subsequent commands and operations interact with the correct Google Cloud project and within the specified geographical region. This will then prevent resources from being created in unintended locations. Second is reproducibility, by explicitly setting these values, you make the notebook's execution more reproducible. Anyone who is running the notebook will use the same project and region, leading to consistent results. Third is avoiding errors. Many googlecloud and BigQuery commands rely on the active project being set. If it is not set, or set incorrectly, commands will fail with errors like "Permission denied" or "Not Found". Similarly, operations like bucket creation or dataset creation requre a region, and not specifying one can lead to errors or resourcesbeing created.

## 1) Kaggle API — What & Why
Use Kaggle CLI for reproducible downloads. Store `kaggle.json` at `~/.kaggle/kaggle.json` with `0600` permissions to protect secrets.

### Build Prompt
Generate a **single Colab code cell** that:
- Prompts me to upload `kaggle.json`,
- Saves to `~/.kaggle/kaggle.json` with `0600` permissions,
- Prints `kaggle --version`.
Add comments about security and reproducibility.


In [8]:
# EXAMPLE (from LLM) — Kaggle setup (commented)
from google.colab import files
print("Upload your kaggle.json (Kaggle > Account > Create New API Token)")
uploaded = files.upload()

import os
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'wb') as f:
     f.write(uploaded[list(uploaded.keys())[0]])
os.chmod('/root/.kaggle/kaggle.json', 0o600)  # owner-only

# # !kaggle --version

Upload your kaggle.json (Kaggle > Account > Create New API Token)


Saving kaggle (4).json to kaggle (4).json


### Verification Prompt
Generate a one-liner that runs `kaggle --help | head -n 20` to show the CLI is ready.


usage: kaggle [-h] [-v] [-W]
              {competitions,c,datasets,d,kernels,k,models,m,files,f,config}
              ...

options:
  -h, --help            show this help message and exit
  -v, --version         Print the Kaggle API version
  -W, --no-warn         Disable out-of-date API version warning

commands:
  {competitions,c,datasets,d,kernels,k,models,m,files,f,config}
                        Use one of:
                        competitions {list, files, download, submit, submissions, leaderboard}
                        datasets {list, files, download, create, version, init, metadata, status}
                        kernels {list, files, init, push, pull, output, status}
                        models {instances, get, list, init, create, delete, update}
                        models instances {versions, get, files, init, create, delete, update}
                        models instances versions {init, create, download, delete, files}
                        config {view, set,

**Reflection:** Why require strict `0600` permissions on API tokens? What risks are we avoiding?

Requiring strict 0600 persmissions on API tokens means that only the owner of the file has read and write access, and not other user has any permissions. This is a critical security measure to protect your API credentials for the following reasons. First, it will prevent unauthorized access, second, it will reduce risk of compromise, and third it for idempotence and reproducability.

## 2) Download & unzip dataset — What & Why
Keep raw files under `/content/data/raw` for predictable paths and auditing.
**Dataset:** `sayeeduddin/netflix-2025user-behavior-dataset-210k-records`

### Build Prompt
Generate a **Colab code cell** that:
- Creates `/content/data/raw`,
- Downloads the dataset to `/content/data` with Kaggle CLI,
- Unzips into `/content/data/raw` (overwrite OK),
- Lists all CSVs with sizes in a neat table.
Include comments describing each step.


In [10]:
# EXAMPLE (from LLM) — Download & unzip (commented)
!mkdir -p /content/data/raw
!kaggle datasets download -d sayeeduddin/netflix-2025user-behavior-dataset-210k-records -p /content/data
!unzip -o /content/data/*.zip -d /content/data/raw
# List CSV inventory
!ls -lh /content/data/raw/*.csv

Dataset URL: https://www.kaggle.com/datasets/sayeeduddin/netflix-2025user-behavior-dataset-210k-records
License(s): CC0-1.0
  0% 0.00/4.02M [00:00<?, ?B/s]
100% 4.02M/4.02M [00:00<00:00, 779MB/s]
Archive:  /content/data/netflix-2025user-behavior-dataset-210k-records.zip
  inflating: /content/data/raw/README.md  
  inflating: /content/data/raw/movies.csv  
  inflating: /content/data/raw/recommendation_logs.csv  
  inflating: /content/data/raw/reviews.csv  
  inflating: /content/data/raw/search_logs.csv  
  inflating: /content/data/raw/users.csv  
  inflating: /content/data/raw/watch_history.csv  
-rw-r--r-- 1 root root 114K Aug  2 19:36 /content/data/raw/movies.csv
-rw-r--r-- 1 root root 4.5M Aug  2 19:36 /content/data/raw/recommendation_logs.csv
-rw-r--r-- 1 root root 1.8M Aug  2 19:36 /content/data/raw/reviews.csv
-rw-r--r-- 1 root root 2.2M Aug  2 19:36 /content/data/raw/search_logs.csv
-rw-r--r-- 1 root root 1.6M Aug  2 19:36 /content/data/raw/users.csv
-rw-r--r-- 1 root root 8.9M A

### Verification Prompt
Generate a snippet that asserts there are exactly **six** CSV files and prints their names.


In [11]:
import glob
csv_files = glob.glob('/content/data/raw/*.csv')
assert len(csv_files) == 6, f"Expected 6 CSV files, but found {len(csv_files)}"
print("Found the following CSV files:")
for csv_file in csv_files:
    print(csv_file)

Found the following CSV files:
/content/data/raw/movies.csv
/content/data/raw/watch_history.csv
/content/data/raw/search_logs.csv
/content/data/raw/users.csv
/content/data/raw/recommendation_logs.csv
/content/data/raw/reviews.csv


**Reflection:** Why is keeping a clean file inventory (names, sizes) useful downstream?

Keeping a clean file inventory is useful for different reasons. First, it allows you to quickly verify that all expected files were downloaded correctly and that their sizes are as anticipated. This is a basic but crucial data quality check. Second, when an issue arises later in the pipeline, having a clear inventory of the raw files is invaluable for troubleshooting. You can confirm the source files exist and haven't been accidentally modified or corrupted. Third, the file inventory serves as documentation of the raw data used. This is essential for reproducibility, allowing others to understand exactly which files were used as input for the rest of the pipeline.

## 3) Create GCS bucket & upload — What & Why
Stage in GCS → consistent, versionable source for BigQuery loads. Bucket names must be **globally unique**.

### Build Prompt
Generate a **Colab code cell** that:
- Creates a unique bucket in `${REGION}` (random suffix),
- Saves name to `BUCKET_NAME` env var,
- Uploads all CSVs to `gs://$BUCKET_NAME/netflix/`,
- Prints the bucket name and explains staging benefits.


In [22]:
# # EXAMPLE (from LLM) — GCS staging (commented)
import uuid, os
bucket_name = f"mgmt467-netflix-{uuid.uuid4().hex[:8]}"
os.environ["BUCKET_NAME"] = bucket_name
# print(f"Using region: {os.environ['REGION']}") # Commented out the print statement
!gcloud storage buckets create gs://$BUCKET_NAME --location=us-central1
!gcloud storage cp /content/data/raw/* gs://$BUCKET_NAME/netflix/
print("Bucket:", bucket_name)
# # # Verify contents
!gcloud storage ls gs://$BUCKET_NAME/netflix/

Creating gs://mgmt467-netflix-99e46d44/...
Copying file:///content/data/raw/movies.csv to gs://mgmt467-netflix-99e46d44/netflix/movies.csv
Copying file:///content/data/raw/README.md to gs://mgmt467-netflix-99e46d44/netflix/README.md
Copying file:///content/data/raw/recommendation_logs.csv to gs://mgmt467-netflix-99e46d44/netflix/recommendation_logs.csv
Copying file:///content/data/raw/reviews.csv to gs://mgmt467-netflix-99e46d44/netflix/reviews.csv
Copying file:///content/data/raw/search_logs.csv to gs://mgmt467-netflix-99e46d44/netflix/search_logs.csv
Copying file:///content/data/raw/users.csv to gs://mgmt467-netflix-99e46d44/netflix/users.csv
Copying file:///content/data/raw/watch_history.csv to gs://mgmt467-netflix-99e46d44/netflix/watch_history.csv

Average throughput: 179.1MiB/s
Bucket: mgmt467-netflix-99e46d44
gs://mgmt467-netflix-99e46d44/netflix/README.md
gs://mgmt467-netflix-99e46d44/netflix/movies.csv
gs://mgmt467-netflix-99e46d44/netflix/recommendation_logs.csv
gs://mgmt467-

In [16]:
import os
print(os.environ["REGION"])

None
us-central1


In [17]:
!gcloud auth login


You are running on a Google Compute Engine virtual machine.
It is recommended that you use service accounts for authentication.

You can run:

  $ gcloud config set account `ACCOUNT`

to switch accounts if necessary.

Your credentials may be visible to others with access to this
virtual machine. Are you sure you want to authenticate with
your personal account?

Do you want to continue (Y/n)?  y

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.

### Verification Prompt
Generate a snippet that lists the `netflix/` prefix and shows object sizes.


In [23]:
import os
!gcloud storage ls -l gs://{os.environ['BUCKET_NAME']}/netflix/

      8002  2025-10-26T22:58:31Z  gs://mgmt467-netflix-99e46d44/netflix/README.md
    115942  2025-10-26T22:58:31Z  gs://mgmt467-netflix-99e46d44/netflix/movies.csv
   4695557  2025-10-26T22:58:31Z  gs://mgmt467-netflix-99e46d44/netflix/recommendation_logs.csv
   1861942  2025-10-26T22:58:31Z  gs://mgmt467-netflix-99e46d44/netflix/reviews.csv
   2250902  2025-10-26T22:58:31Z  gs://mgmt467-netflix-99e46d44/netflix/search_logs.csv
   1606820  2025-10-26T22:58:31Z  gs://mgmt467-netflix-99e46d44/netflix/users.csv
   9269425  2025-10-26T22:58:31Z  gs://mgmt467-netflix-99e46d44/netflix/watch_history.csv
TOTAL: 7 objects, 19808590 bytes (18.89MiB)


**Reflection:** Name two benefits of staging in GCS vs loading directly from local Colab.

Two benefits are scalability and performance, and reliability and durability. GCS is a hgihly scalable and performant object storage service. Loading it into BigQuery directly from GCS is significanatly faster and more efficient, especially for large datasets, because BigQuery can read data parallel directly from GCS. Also, data stored in GCS is highly durable and available, it's stored redundantly across multiple locations to protect against data loss.

## 4) BigQuery dataset & loads — What & Why
Create dataset `netflix` and load six CSVs with **autodetect** for speed (we’ll enforce schemas later).

### Build Prompt (two cells)
**Cell A:** Create (idempotently) dataset `netflix` in US multi-region; if it exists, print a friendly message.  
**Cell B:** Load tables from `gs://$BUCKET_NAME/netflix/`:
`users, movies, watch_history, recommendation_logs, search_logs, reviews`
with `--skip_leading_rows=1 --autodetect --source_format=CSV`.
Finish with row-count queries for each table.


In [24]:
# # EXAMPLE (from LLM) — BigQuery dataset (commented)
DATASET="netflix"
# # # Attempt to create; ignore if exists
!bq --location=US mk -d --description "MGMT467 Netflix dataset" $DATASET || echo "Dataset may already exist."

BigQuery error in mk operation: Dataset 'mgmt-467-35946:netflix' already exists.
Dataset may already exist.


In [30]:
#EXAMPLE (from LLM) — Load tables (commented)
tables = {
  "users": "users.csv",
  "movies": "movies.csv",
  "watch_history": "watch_history.csv",
  "recommendation_logs": "recommendation_logs.csv",
  "search_logs": "search_logs.csv",
  "reviews": "reviews.csv",
}
import os
from google.cloud import bigquery

# Define DATASET here
DATASET = "netflix"

# Initialize BigQuery client
client = bigquery.Client(project=os.environ['GOOGLE_CLOUD_PROJECT'])

for tbl, fname in tables.items():
  src = f"gs://{os.environ['BUCKET_NAME']}/netflix/{fname}"
  # Corrected destination format
  dest = f"{DATASET}.{tbl}"
  print("Loading", tbl, "from", src)
  # Construct the bq load command explicitly
  load_command = f"bq load --source_format=CSV --skip_leading_rows=1 --autodetect {dest} {src}"
  !{load_command}

# Row counts using BigQuery client library
print("\nRunning row count queries:")
for tbl in tables.keys():
  # Construct the SQL query string in Python
  sql_query = f"SELECT '{tbl}' AS table_name, COUNT(*) AS n FROM `{os.environ['GOOGLE_CLOUD_PROJECT']}.{DATASET}.{tbl}`"
  print(f"  Querying table: {tbl}")
  try:
    query_job = client.query(sql_query)
    results = query_job.result()
    for row in results:
      print(f"    {row.table_name}: {row.n} rows")
  except Exception as e:
    print(f"    Error querying table {tbl}: {e}")

Loading users from gs://mgmt467-netflix-99e46d44/netflix/users.csv
Waiting on bqjob_r5784c2f536e8b8a2_0000019a22eaf8c8_1 ... (2s) Current status: DONE   
Loading movies from gs://mgmt467-netflix-99e46d44/netflix/movies.csv
Waiting on bqjob_r4f54a5c8908a0a30_0000019a22eb1070_1 ... (1s) Current status: DONE   
Loading watch_history from gs://mgmt467-netflix-99e46d44/netflix/watch_history.csv
Waiting on bqjob_r1747cad9f728dc47_0000019a22eb26de_1 ... (2s) Current status: DONE   
Loading recommendation_logs from gs://mgmt467-netflix-99e46d44/netflix/recommendation_logs.csv
Waiting on bqjob_r6635550104a1cc6b_0000019a22eb3f34_1 ... (1s) Current status: DONE   
Loading search_logs from gs://mgmt467-netflix-99e46d44/netflix/search_logs.csv
Waiting on bqjob_r7b5b2b12c44c7446_0000019a22eb54b3_1 ... (1s) Current status: DONE   
Loading reviews from gs://mgmt467-netflix-99e46d44/netflix/reviews.csv
Waiting on bqjob_rbb8f2adbb3e836_0000019a22eb6904_1 ... (1s) Current status: DONE   

Running row cou

## 5) Data Quality (DQ) — Concepts we care about
- **Missingness** (MCAR/MAR/MNAR). Impute vs drop. Add `is_missing_*` indicators.
- **Duplicates** (exact vs near). Double-counted engagement corrupts labels & KPIs.
- **Outliers** (IQR). Winsorize/cap vs robust models. Always **flag** and explain.
- **Reproducibility**. Prefer `CREATE OR REPLACE` and deterministic keys.


### 5.1 Missingness (users) — What & Why
Measure % missing and check if missingness depends on another variable (MAR) → potential bias & instability.

### Build Prompt
Generate **two BigQuery SQL cells** that:
1. Total rows and % missing in `region`, `plan_tier`, `age_band` from `users`.
2. `% plan_tier missing by region` ordered descending. Add comments on MAR.

In [41]:
import os
from google.cloud import bigquery

project_id = os.environ['GOOGLE_CLOUD_PROJECT']
client = bigquery.Client(project=project_id)

dataset_id = "netflix"
table_id = "users"

table_ref = client.dataset(dataset_id).table(table_id)
table = client.get_table(table_ref)

print(f"Schema for table {project_id}.{dataset_id}.{table_id}:")
for field in table.schema:
    print(f"- {field.name}: {field.field_type}")

Schema for table mgmt-467-35946.netflix.users:
- user_id: STRING
- email: STRING
- first_name: STRING
- last_name: STRING
- age: FLOAT
- gender: STRING
- country: STRING
- state_province: STRING
- city: STRING
- subscription_plan: STRING
- subscription_start_date: DATE
- is_active: BOOLEAN
- monthly_spend: FLOAT
- primary_device: STRING
- household_size: FLOAT
- created_at: TIMESTAMP


In [46]:
# Total rows and % missing in region, plan_tier, age_band from users
%%bigquery
SELECT
    COUNT(*) AS total_rows,
    COUNTIF(state_province IS NULL) AS missing_region,
    ROUND(COUNTIF(state_province IS NULL) * 100 / COUNT(*), 2) AS percent_missing_region,
    COUNTIF(subscription_plan IS NULL) AS missing_plan_tier,
    ROUND(COUNTIF(subscription_plan IS NULL) * 100 / COUNT(*), 2) AS percent_missing_plan_tier,
    COUNTIF(age IS NULL) AS missing_age_band,
    ROUND(COUNTIF(age IS NULL) * 100 / COUNT(*), 2) AS percent_missing_age_band
FROM
    `mgmt-467-35946.netflix.users`

Query is running:   0%|          |

Downloading:   0%|          |

,total_rows,missing_region,percent_missing_region,missing_plan_tier,percent_missing_plan_tier,missing_age_band,percent_missing_age_band
0,61800,0,0.0,0,0.0,7374,11.93


In [50]:
# % plan_tier missing by region ordered descending (MAR)
%%bigquery
SELECT
    state_province,
    COUNT(*) AS n,
    ROUND(COUNTIF(subscription_plan IS NULL) * 100 / COUNT(*), 2) AS pct_missing_plan_tier_by_region
FROM
    `mgmt-467-35946.netflix.users`
GROUP BY
    state_province
ORDER BY
    pct_missing_plan_tier_by_region DESC

Query is running:   0%|          |

Downloading:   0%|          |

,state_province,n,pct_missing_plan_tier_by_region
0,Arizona,2166,0.0
1,California,2178,0.0
2,Florida,2118,0.0
3,Illinois,2130,0.0
4,Indiana,2244,0.0
5,New Brunswick,1728,0.0
6,North Carolina,2496,0.0
7,Nova Scotia,2046,0.0
8,Pennsylvania,2130,0.0
9,Tennessee,2358,0.0


### Verification Prompt
Generate a single query that returns `table_name, row_count` for all six tables in `${GOOGLE_CLOUD_PROJECT}.netflix`.


In [54]:
# Generate a single query that returns table_name, row_count for all six tables
import os
project_id = os.environ['GOOGLE_CLOUD_PROJECT']

In [58]:
# Generate a single query that returns table_name, row_count for all six tables
import os
from google.cloud import bigquery

project_id = os.environ['GOOGLE_CLOUD_PROJECT']
client = bigquery.Client(project=project_id)
dataset_id = "netflix"

print(f"Tables in dataset {project_id}.{dataset_id} and their row counts:")
tables = client.list_tables(dataset_id)  # Make an API request.

for table in tables:
    # Retrieve the table to get its properties (including num_rows)
    table_ref = client.dataset(dataset_id).table(table.table_id)
    table = client.get_table(table_ref)
    print(f"- {table.table_id}: {table.num_rows} rows")

Tables in dataset mgmt-467-35946.netflix and their row counts:
- movies: 6240 rows
- recomendation_logs: 52000 rows
- recommendation_logs: 260000 rows
- reviews: 92700 rows
- search_logs: 159000 rows
- users: 61800 rows
- watch-history: 105000 rows
- watch_history: 525000 rows


**Reflection:** When is `autodetect` acceptable? When should you enforce explicit schemas and why?

Autodetect is acceptable when you are first exploring a new dataset and don't have a clear understanding of its structure, autodetect is a quick way to get the data into BigQuery and start querying. It is good also for Ad-hoc Analysis, for on-off analyses wherer strict data typing ins't critical, autodetect can save time.

**Reflection:** Which columns are most missing? Hypothesize MCAR/MAR/MNAR and why.

It looks like the column with the most missing values is age, region and subscripton_plan seem to have 0% missing values. MCAR, the missingness of age is completely unrelated to any other variable in the dataset. If it were MCAR, the missing values would be purely by chance, this is less likely for a demographic field like age unless there was a random data collection error. It is more plausible that the missingness in the age column is MNAR. Users might be less inclined to share their age if they fall into certain age groups, or if they have privacy conerns related to their age.

### 5.2 Duplicates (watch_history) — What & Why
Find exact duplicate interaction records and keep **one best** per group (deterministic policy).

### Build Prompt
Generate **two BigQuery SQL cells**:
1) Report duplicate groups on `(user_id, movie_id, event_ts, device_type)` with counts (top 20).
2) Create table `watch_history_dedup` that keeps one row per group (prefer higher `progress_ratio`, then `minutes_watched`). Add comments.


In [65]:
# Report duplicate groups on (user_id, movie_id, event_ts, device_type) with counts (top 20)
import os
project_id = os.environ['GOOGLE_CLOUD_PROJECT']

In [70]:
%%bigquery --project {project_id}
SELECT
    user_id,
    movie_id,
    watch_date,
    device_type,
    COUNT(*) AS dup_count
FROM
    `mgmt-467-35946.netflix.watch_history`
GROUP BY
    user_id,
    movie_id,
    watch_date,
    device_type
HAVING
    COUNT(*) > 1
ORDER BY
    dup_count DESC
LIMIT 20

Query is running:   0%|          |

Downloading:   0%|          |

,user_id,movie_id,watch_date,device_type,dup_count
0,user_00391,movie_0893,2024-08-26,Laptop,20
1,user_03310,movie_0640,2024-09-08,Smart TV,20
2,user_01870,movie_0844,2024-06-02,Laptop,15
3,user_04513,movie_0564,2024-06-11,Mobile,15
4,user_08681,movie_0332,2024-06-13,Laptop,15
5,user_06554,movie_0505,2025-10-02,Laptop,15
6,user_07594,movie_0133,2025-03-24,Laptop,15
7,user_05404,movie_0691,2025-01-01,Mobile,15
8,user_04899,movie_0142,2025-01-20,Desktop,15
9,user_01383,movie_0015,2025-04-29,Desktop,15


In [73]:
#EXAMPLE (from LLM) — Detect duplicate groups (commented)
%%bigquery --project {project_id}
SELECT user_id, movie_id, watch_date, device_type, COUNT(*) AS dup_count
FROM `mgmt-467-35946.netflix.watch_history`
GROUP BY user_id, movie_id, watch_date, device_type
HAVING dup_count > 1
ORDER BY dup_count DESC
LIMIT 20;

Query is running:   0%|          |

Downloading:   0%|          |

,user_id,movie_id,watch_date,device_type,dup_count
0,user_00391,movie_0893,2024-08-26,Laptop,20
1,user_03310,movie_0640,2024-09-08,Smart TV,20
2,user_00928,movie_0913,2024-01-18,Laptop,15
3,user_02126,movie_0642,2025-02-09,Desktop,15
4,user_02284,movie_0914,2024-12-30,Laptop,15
5,user_09331,movie_0073,2024-03-23,Smart TV,15
6,user_01226,movie_0710,2024-05-30,Smart TV,15
7,user_03094,movie_0114,2024-05-13,Smart TV,15
8,user_01182,movie_0794,2025-07-03,Desktop,15
9,user_09973,movie_0342,2025-03-22,Desktop,15


In [79]:
## EXAMPLE (from LLM) — Keep-one policy (commented)
%%bigquery --project {project_id}
CREATE OR REPLACE TABLE `mgmt-467-35946.netflix.watch_history_dedup` AS
SELECT * EXCEPT(rk) FROM (
  SELECT h.*,
         ROW_NUMBER() OVER (
           PARTITION BY user_id, movie_id, watch_date, device_type
           ORDER BY progress_percentage DESC, watch_duration_minutes DESC
         ) AS rk
  FROM `mgmt-467-35946.netflix.watch_history` h
)
WHERE rk = 1;

Query is running:   0%|          |

""


### Verification Prompt
Generate a before/after count query comparing raw vs `watch_history_dedup`.


In [85]:
# Generate a before/after count query comparing raw vs watch_history_dedup
import os
project_id = os.environ['GOOGLE_CLOUD_PROJECT']

In [86]:
%%bigquery --project {project_id}
SELECT 'watch_history_raw' AS table_name, COUNT(*) AS row_count FROM `mgmt-467-35946.netflix.watch_history`
UNION ALL
SELECT 'watch_history_dedup' AS table_name, COUNT(*) AS row_count FROM `mgmt-467-35946.netflix.watch_history_dedup`

Query is running:   0%|          |

Downloading:   0%|          |

,table_name,row_count
0,watch_history_dedup,100000
1,watch_history_raw,525000


**Reflection:** Why do duplicates arise (natural vs system-generated)? How do they corrupt labels and KPIs?

Duplicates can arise for different reasons. There are Natural Duplicates, which occur when real-world entities or events are legitametely recorded multiple times, but each recording represents the same underlying thing. It can also be a system-generated duplicate, which are unintended duplicates introduced by technical processes or errors in the data ipeline. This is the more problematic type of duplicate. Duplicates can corrupt labels and KPIs trhough inflated counts, biased aggregations, and incorrect labels for machine learning.

### 5.3 Outliers (minutes_watched) — What & Why
Estimate extreme values via IQR; report % outliers; **winsorize** to P01/P99 for robustness while also **flagging** extremes.

### Build Prompt
Generate **two BigQuery SQL cells**:
1) Compute IQR bounds for `minutes_watched` on `watch_history_dedup` and report % outliers.
2) Create `watch_history_robust` with `minutes_watched_capped` capped at P01/P99; return quantile summaries before/after.


In [88]:
##EXAMPLE (from LLM) — IQR outlier rate (commented)
%%bigquery --project {project_id}
WITH dist AS (
  SELECT
    APPROX_QUANTILES(watch_duration_minutes, 4)[OFFSET(1)] AS q1,
    APPROX_QUANTILES(watch_duration_minutes, 4)[OFFSET(3)] AS q3
  FROM `mgmt-467-35946.netflix.watch_history_dedup`
),
bounds AS (
  SELECT q1, q3, (q3-q1) AS iqr,
         q1 - 1.5*(q3-q1) AS lo,
         q3 + 1.5*(q3-q1) AS hi
  FROM dist
)
SELECT
  COUNTIF(h.watch_duration_minutes < b.lo OR h.watch_duration_minutes > b.hi) AS outliers,
  COUNT(*) AS total,
  ROUND(100*COUNTIF(h.watch_duration_minutes < b.lo OR h.watch_duration_minutes > b.hi)/COUNT(*),2) AS pct_outliers
FROM `mgmt-467-35946.netflix.watch_history_dedup` h
CROSS JOIN bounds b;

Query is running:   0%|          |

Downloading:   0%|          |

,outliers,total,pct_outliers
0,3472,100000,3.47


In [90]:
##EXAMPLE (from LLM) — Winsorize + quantiles (commented)
%%bigquery --project {project_id}

CREATE OR REPLACE TABLE `mgmt-467-35946.netflix.watch_history_robust` AS
WITH q AS (
  SELECT
    APPROX_QUANTILES(watch_duration_minutes, 100)[OFFSET(1)]  AS p01,
    APPROX_QUANTILES(watch_duration_minutes, 100)[OFFSET(98)] AS p99
  FROM `mgmt-467-35946.netflix.watch_history_dedup`
)
SELECT
  h.*,
  GREATEST(q.p01, LEAST(q.p99, h.watch_duration_minutes)) AS watch_duration_minutes_capped
FROM `mgmt-467-35946.netflix.watch_history_dedup` h, q;

-- Quantiles before vs after
WITH before AS (
  SELECT 'before' AS which, APPROX_QUANTILES(watch_duration_minutes, 5) AS q
  FROM `mgmt-467-35946.netflix.watch_history_dedup`
),
after AS (
  SELECT 'after' AS which, APPROX_QUANTILES(watch_duration_minutes_capped, 5) AS q
  FROM `mgmt-467-35946.netflix.watch_history_robust`
)
SELECT * FROM before UNION ALL SELECT * FROM after;

Query is running:   0%|          |

Downloading:   0%|          |

,which,q
0,before,"[0.2, 24.9, 41.7, 61.2, 91.7, 799.3]"
1,after,"[4.4, 24.6, 41.5, 61.5, 92.0, 203.6]"


### Verification Prompt
Generate a query that shows min/median/max before vs after capping.


In [ ]:
# Generate a query that shows min/median/max before vs after capping
import os
project_id = os.environ['GOOGLE_CLOUD_PROJECT']

In [93]:
%%bigquery --project {project_id}
SELECT
    'Before Capping' AS metric,
    MIN(watch_duration_minutes) AS min_duration,
    APPROX_QUANTILES(watch_duration_minutes, 100)[OFFSET(50)] AS median_duration,
    MAX(watch_duration_minutes) AS max_duration
FROM
    `mgmt-467-35946.netflix.watch_history_robust`
UNION ALL
SELECT
    'After Capping' AS metric,
    MIN(watch_duration_minutes_capped) AS min_duration,
    APPROX_QUANTILES(watch_duration_minutes_capped, 100)[OFFSET(50)] AS median_duration,
    MAX(watch_duration_minutes_capped) AS max_duration
FROM
    `mgmt-467-35946.netflix.watch_history_robust`

Query is running:   0%|          |

Downloading:   0%|          |

,metric,min_duration,median_duration,max_duration
0,Before Capping,0.2,51.2,799.3
1,After Capping,4.4,51.2,203.6


**Reflection:** When might capping be harmful? Name a model type less sensitive to outliers and why.

Capping outliers, while useful for some models, can sometimes be harmful, and certain model types are inherently less sensitive to extreme values. Capping can be harmful in cases like loss of information, because capping compresses all values outside the capping range, to a single value, this can lead to a loss of potentially valuable information contained in the extreme values. It can also affect distortion of data distribution, capping changes the shape of the shape of the data distribution by creating a spike at the capping value.

### 5.4 Business anomaly flags — What & Why
Human-readable flags help both product decisioning and ML features (e.g., binge behavior).

### Build Prompt
Generate **three BigQuery SQL cells** (adjust if columns differ):
1) In `watch_history_robust`, compute and summarize `flag_binge` for sessions > 8 hours.
2) In `users`, compute and summarize `flag_age_extreme` if age can be parsed from `age_band` (<10 or >100).
3) In `movies`, compute and summarize `flag_duration_anomaly` where `duration_min` < 15 or > 480 (if exists).
Each cell should output count and percentage and include 1–2 comments.


In [94]:
#EXAMPLE (from LLM) — flag_binge (commented)
%%bigquery --project {project_id}

SELECT
  COUNTIF(watch_duration_minutes > 8*60) AS sessions_over_8h,
  COUNT(*) AS total,
  ROUND(100*COUNTIF(watch_duration_minutes > 8*60)/COUNT(*),2) AS pct
FROM `mgmt-467-35946.netflix.watch_history_robust`;

Query is running:   0%|          |

Downloading:   0%|          |

,sessions_over_8h,total,pct
0,639,100000,0.64


In [ ]:
# In users, compute and summarize flag_age_extreme if age < 10 or > 100
import os
project_id = os.environ['GOOGLE_CLOUD_PROJECT']

In [96]:
%%bigquery --project {project_id}
SELECT
  COUNTIF(age IS NOT NULL AND (age < 10 OR age > 100)) AS extreme_age_users,
  COUNT(*) AS total_users,
  ROUND(COUNTIF(age IS NOT NULL AND (age < 10 OR age > 100)) * 100 / COUNT(*), 2) AS pct_extreme_age
FROM
  `mgmt-467-35946.netflix.users`;

Query is running:   0%|          |

Downloading:   0%|          |

,extreme_age_users,total_users,pct_extreme_age
0,1074,61800,1.74


In [100]:
#EXAMPLE (from LLM) — flag_duration_anomaly (commented)
%%bigquery --project {project_id}

SELECT
  COUNTIF(duration_minutes < 15) AS titles_under_15m,
  COUNTIF(duration_minutes > 8*60) AS titles_over_8h,
  COUNT(*) AS total
FROM `mgmt-467-35946.netflix.movies`;

Query is running:   0%|          |

Downloading:   0%|          |

,titles_under_15m,titles_over_8h,total
0,72,66,6240


### Verification Prompt
Generate a single compact summary query that returns two columns per flag: `flag_name, pct_of_rows`.


In [ ]:
# Generate a single compact summary query that returns two columns per flag: flag_name, pct_of_rows.
import os
project_id = os.environ['GOOGLE_CLOUD_PROJECT']

In [102]:
%%bigquery --project {project_id}
SELECT
    'flag_binge' AS flag_name,
    ROUND(COUNTIF(watch_duration_minutes > 8*60) * 100 / COUNT(*), 2) AS pct_of_rows
FROM
    `mgmt-467-35946.netflix.watch_history_robust`
UNION ALL
SELECT
    'flag_age_extreme' AS flag_name,
    ROUND(COUNTIF(age IS NOT NULL AND (age < 10 OR age > 100)) * 100 / COUNT(*), 2) AS pct_of_rows
FROM
    `mgmt-467-35946.netflix.users`
UNION ALL
SELECT
    'flag_duration_anomaly' AS flag_name,
    ROUND(COUNTIF(duration_minutes < 15 OR duration_minutes > 480) * 100 / COUNT(*), 2) AS pct_of_rows
FROM
    `mgmt-467-35946.netflix.movies`

Query is running:   0%|          |

Downloading:   0%|          |

,flag_name,pct_of_rows
0,flag_binge,0.64
1,flag_age_extreme,1.74
2,flag_duration_anomaly,2.21


**Reflection:** Which anomaly flag is most common? Which would you keep as a feature and why?

*   The most common anomaly is the flag_duration_anomaly which affects 2.21% of the movies
*.  The second most common anomaly would be the flag_age_extreme which affects 1.74% of users.

I think that I would keep flag_bing, I think that it directly captures a specific user behaviour. This is often a strong signal for user engagement, content preference, or potential churn risk, which can be highly relevant for recomendation systems, user segmentaation, or predicting future behaviour.



## 6) Save & submit — What & Why
Reproducibility: save artifacts and document decisions so others can rerun and audit.

### Build Prompt
Generate a checklist (Markdown) students can paste at the end:
- Save this notebook to the team Drive.
- Export a `.sql` file with your DQ queries and save to repo.
- Push notebook + SQL to the **team GitHub** with a descriptive commit.
- Add a README with your `PROJECT_ID`, `REGION`, bucket, dataset, and today’s row counts.


## Grading rubric (quick)
- Profiling completeness (30)  
- Cleaning policy correctness & reproducibility (40)  
- Reflection/insight (20)  
- Hygiene (naming, verification, idempotence) (10)
